<a href="https://colab.research.google.com/github/LauraJouLee/Fall2021-TermProject-DigitalHumanities/blob/main/Step%203%20Merge%20with%20COVID-19%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
billboardhot100 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DH_Final_Hot100.csv')
spotifyerror = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Spotify_error_track_merged.csv')
spotifynonerror = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Spotify_nonerror_track_merged_2.csv')

len(list(spotifyerror['id'])) + len(list(spotifynonerror['id'])) #, len(list(billboardhot100['link']))


In [ ]:
r = billboardhot100.loc[billboardhot100['rank'] == 68]
r.loc[r['date']== '2021-09-25']

In [ ]:
spotifynonerror= spotifynonerror.rename(columns={'track_list': 'track'})
#spotifyerror.columns, spotifynonerror.columns

Spotify_audio_features = pd.concat([spotifyerror, spotifynonerror], axis=0).drop(columns='Unnamed: 0')
Spotify_audio_features.drop_duplicates()#.to_csv('Spotify_audio_features.csv')

Spotify_audio_features.loc[Spotify_audio_features['track'] == 'Better Days']


In [ ]:
billboardhot100.merge(Spotify_audio_features.drop_duplicates(), left_on='song', right_on='track', how='inner')

# **Data Cleaning**
### Check Billboard Duplicate

In [ ]:
#generate new artist column without specific char

toBeReplace = ['(', ':', '.', ', ']
song_replaced = []
l=[]

for a in list(billboardhot100['song']):
  song_replaced.append(a)

for elem in toBeReplace:
    # Check if string is in the main string
    for k,v in enumerate(song_replaced):
        if elem in v:
            # Replace the string
            song_replaced[k] = v.split(elem)[0]

[l.append(x.lower()) for x in song_replaced]

billboardhot100['track_replaced_lowercase']= l
len(billboardhot100)

In [ ]:
billboardhot100 = billboardhot100.drop(columns=['Unnamed: 0'])
spotifyerror = spotifyerror.drop(columns=['Unnamed: 0'])
spotifynonerror = spotifynonerror.drop(columns=['Unnamed: 0'])



In [ ]:
spotifyerror_nodup = spotifyerror.drop_duplicates().reset_index().drop(columns=['index'])
spotifynonerror_nodup = spotifynonerror.drop_duplicates().reset_index().drop(columns=['index'])
spotify_nodup = pd.concat([spotifyerror_nodup,spotifynonerror_nodup], axis=0).reset_index()

In [ ]:
track_replaced=[]
l2=[]

for i in list(spotify_nodup['track'].apply(str)):
  track_replaced.append(i)

for elem in toBeReplace:
    # Check if string is in the main string
    for k,v in enumerate(track_replaced):
        if elem in v:
            # Replace the string
            track_replaced[k] = v.split(elem)[0]

[l2.append(x.lower()) for x in track_replaced]

spotify_nodup['track_replaced_lowercase']= l2
len(spotify_nodup)

In [ ]:
spotify_nodup

# **Data Cleaning**
### Clean original data from Our World of Data and select the needed columns
### Original Dataset Github Link: https://github.com/owid/covid-19-data/blob/285623e2b71d0039f1329b6002bc9c202d279281/public/data/owid-covid-data.csv

In [ ]:
covid_daily = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/owid-covid-data.csv')
covid_daily = covid_daily[covid_daily['iso_code']=='USA'].reset_index()
covid_daily = covid_daily.drop(['index','iso_code','continent','location'], axis=1)
covid = covid_daily.dropna(axis=1, how='all')

In [ ]:
covid['date'] = pd.to_datetime(covid['date'])
covid1 = covid.resample('W', on='date')['new_cases','new_deaths_smoothed','new_tests_smoothed','people_fully_vaccinated','new_vaccinations'].sum().reset_index()


In [ ]:
covid1['date'] = covid1['date'] - pd.to_timedelta(1, unit='d')

In [ ]:
covid1.to_csv('covid_by_week.csv')

# **Data Analysis**
### Linear Regression

In [ ]:
import statistics
import datetime

clean0 = pd.read_excel('/content/BB&Spotify.xlsx')
clean = clean0.groupby('date').mean().drop(columns=['rank']).merge(covid1, on='date', how='left')

clean['year']= clean['date'].dt.year
clean['month']= clean['date'].dt.month
clean['week_number'] = clean['date'].dt.week

model = clean.drop(['date'], axis=1)


In [ ]:
import statsmodels.formula.api as sm

r = sm.ols('valence ~ new_cases	+ new_vaccinations + month*year ', data=model).fit()
result = r.summary()

result